In [ ]:
from __future__ import print_function

import numpy as np
import tensorflow as tf
from tensorflow.contrib import rnn

# Training Parameters
params = {
    'learning_rate': 0.001,
    'training_steps': 20000,
    'batch_size': 128,
    'display_step': 100,

    # Network Parameters
    'num_input': 28,  # MNIST data input (img shape: 28*28)
    'timesteps': 28,  # timesteps
    'num_hidden': 128,  # hidden layer num of features
    'num_classes': 10  # MNIST total classes (0-9 digits)
}

def estimator_spec(logits, labels, mode, params):
    loss, train_op, predictions = None, None, None
    if mode == tf.estimator.ModeKeys.TRAIN or mode == tf.estimator.ModeKeys.EVAL:
        onehot_labels = tf.one_hot(labels, params['num_classes'], 1, 0)
        loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=onehot_labels))

    if mode == tf.estimator.ModeKeys.TRAIN:
        optimizer = tf.train.GradientDescentOptimizer(learning_rate=params['learning_rate'])
        train_op = optimizer.minimize(loss, global_step=tf.train.get_global_step())

    if mode == tf.estimator.ModeKeys.PREDICT:
        predictions = {'prediction': tf.argmax(logits, 1),
                       'prob': tf.nn.softmax(logits)}
    eval_metric_ops = {'accuracy': tf.metrics.accuracy(labels=labels, predictions=tf.argmax(logits, 1))}
    return tf.estimator.EstimatorSpec(
        mode=mode,
        predictions=predictions,
        loss=loss,
        train_op=train_op,
        eval_metric_ops=eval_metric_ops)

def model_fn(features, labels, mode, params):
    '''
    To classify images using a recurrent neural network, we consider every image
    row as a sequence of pixels. Because MNIST image shape is 28*28px, we will then
    handle 28 sequences of 28 steps for every sample.
    '''

    x = tf.reshape(features['x'], [-1, params['timesteps'], params['num_input']])
    # Unstack to get a list of 'timesteps' tensors of shape (batch_size, n_input)
    x = tf.unstack(x, params['timesteps'], 1)

    # Define a lstm cell with tensorflow
    lstm_cell = rnn.BasicLSTMCell(params['num_hidden'], forget_bias=1.0)

    # Get lstm cell output
    outputs, states = rnn.static_rnn(lstm_cell, x, dtype=tf.float32)
    # outputs, states = tf.nn.dynamic_rnn(lstm_cell, x, dtype=tf.float32)

    # Linear activation, using rnn inner loop last output
    logits = tf.layers.dense(states[-1], units=params['num_classes'])
    return estimator_spec(logits=logits, labels=labels, mode=mode, params=params)

def main(args = None):
    run_config = tf.estimator.RunConfig()
    # run_config = run_config.replace(**{'save_checkpoints_steps': 100, 'keep_checkpoint_max': 20})
    classifier = tf.estimator.Estimator(model_fn=model_fn,
                                        model_dir='drive/tmp/checkpoint',
                                        params=params,
                                        config=run_config)

    # make data
    mnist = tf.contrib.learn.datasets.DATASETS['mnist']('/tmp/mnist')
    
    train_input_fn = tf.estimator.inputs.numpy_input_fn(
        x = {'x': mnist.train.images},
        y = mnist.train.labels.astype(np.int32),
        batch_size=params['batch_size'],
        num_epochs=None,
        shuffle=True)

    eval_input_fn = tf.estimator.inputs.numpy_input_fn(
        x={'x': mnist.train.images},
        y=mnist.train.labels.astype(np.int32),
        num_epochs=1,
        shuffle=False)

    # classifier.train(input_fn=train_input_fn, steps=params['training_steps'], hooks=[])

    train_spec = tf.estimator.TrainSpec(input_fn=train_input_fn, max_steps=params['training_steps'])
    eval_spec = tf.estimator.EvalSpec(input_fn=eval_input_fn)
    tf.estimator.train_and_evaluate(classifier, train_spec, eval_spec)
    score = classifier.evaluate(eval_input_fn, steps=1)
    print(score)
    
if __name__ == '__main__':
    main(None)